In [4]:
# 標準
import pickle
import os
import sys
sys.path.append('./src/')#モジュールが入っているディレクトリのパスを指定
sys.path.append('./src/module/')
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display

import numpy as np
import pandas as pd
pd.set_option("max_rows", 500, "max_colwidth", 500, "max_columns", 500)
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
from module import config
from module.train import Trainer

from sklearn.model_selection import StratifiedKFold

## 演習1

今回の研修でCVを紹介しました。今までtrain_test_splitだった箇所をStratifiedKFoldに変更して5foldで平均AUCを計算してみて下さい。それが出来たら次に**kfold ensemble**という手法を実装して下さい。これはkfoldによってモデルを5個作ってそれぞれの予測結果を平均するという方法です。イメージとしてはk=5なら

$$
(pred1+pred2+pred3+pred4+pred5)/5
$$

という単純な操作です。これをtestデータに対して実行し、ホールドアウト法で学習・予測してsubmitした結果と比べてどちらがAUCが向上するか確かめてみて下さい。

## train_test_split => StaritifiedKFold

In [13]:
df = pd.read_parquet("/Volumes/ex-ssd/kaggle/data/home_default_risk/dataframe/application_train.parquet")
X, y =  df.drop(columns=config.IGNORE_COLS), df.TARGET

folds = StratifiedKFold(n_splits=config.N_FOLD, shuffle=True, random_state=config.RANDOM_STATE)

In [14]:
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    trainer.fit(X.iloc[train_idx], y.iloc[train_idx])
    score = trainer.evaluate_auc(X.iloc[valid_idx], y.iloc[valid_idx])
    print("{0}fold AUC:".format(n_fold), score)
    scores.append(score)

print('AUC: {0:.4f}±{1:.4f}'.format(np.mean(scores), np.std(scores)))

0fold AUC: 0.747184848397732
1fold AUC: 0.7461518387169246
2fold AUC: 0.7453818379255279
3fold AUC: 0.7466196338623865
4fold AUC: 0.7430567592903763
AUC: 0.7457±0.0014


## kfold ensembleしてsubmit

configの設定に追加
```py
N_FOLD = 5
IS_KFOLD_ENSEMBLE = True

```

In [27]:
!python src/module/preprocessing.py --csv_name "application_train.csv"

starting
read_csv done
preprocessor fit done
df parquet save done
Preprocessor pickle dump done


In [28]:
!python src/module/train.py --parquet_name "application_train.parquet"

starting
read parquet done
0fold AUC: 0.7469214958563992
1fold AUC: 0.7456454765579549
2fold AUC: 0.7450221059416909
3fold AUC: 0.746547836375758
4fold AUC: 0.7425103552662828
AUC: 0.7453±0.0016
save model done


In [29]:
!python src/module/test.py --csv_name "application_test.csv"

starting
read_csv done
pickle load done
   SK_ID_CURR  ...  TARGET
0      100001  ...       0
1      100005  ...       0
2      100013  ...       0
3      100028  ...       0
4      100038  ...       0

[5 rows x 199 columns]
predict done
100%|███████████████████████████████████████| 1.26M/1.26M [00:06<00:00, 203kB/s]
Successfully submitted to Home Credit Default Risksumit done! message: 'kfold unsemble'


## 演習2

では実際に特徴量エンジニアリング、特徴量選択、train_test_split→CVに変更などを実際に試してみて、最低でも10パターンの実験を行って下さい。その際にbaselineは私の配布したスクリプト（スケーリング無のもの）として、それに加える形で行って下さい。

どのような実験をしたか、を管理するのはどのツール使っても構いません。エクセルでも良いし、TrelloでもJIRAでもいいです。gitのコミットメッセージは使いにくいかも。

※今回もLogisticRegression縛りで。スケーリング無にするにはconfig.pyのSCALING = Falseにしてください。

In [55]:
pd.read_csv("./HomeCreditDefaultRiskスコア管理表 - シート1.csv")

,ver,行ったこと,狙い,手元のスコア(VAL or CV),Private Score,Public Score,所感
0,1,数値=>平均埋め and カテゴリ=> OneHot\nロジスティック回帰,比較用のベースラインの作成,train_auc 0.61798\nval_auc 0.62675,0.61071,0.62251,ランダムで0.5だから\n思ったより低い。。
1,2,"ver1から〇〇_AVG, MEDI, MODE\n特徴量を消す",不要そうな特徴量を消して\n効果を見る,train_auc 0.61843\nval_auc 0.627160,0.61134,0.62275,ほんの少し上がったが、\nそもそもスケーリング無しで未学習?
2,3,ver1に標準化を追加する,スケーリング、学習の高速化、\n回帰係数の解釈しやすく,train_auc 0.747766\nval_auc 0.74961,0.73372,0.73848,スケーリングが思ったより大事！\n僅かに過学習
3,4,ver2に標準化を追加する,不要そうな特徴量を消して\n効果を見る,train_auc 0.74726\nval_auc 0.74922,0.73239,0.73843,ほとんど意味ない特徴だが、\n一部は有効なのかもしれない。
4,5,ver4のtrain_test_split を\nkfold ensembleにする,データ分けのランダム依存を\n防ぐ,AUC: 0.7453±0.0016,0.73255,0.73874,思ったより上がらなかった。\nコード間違ってる？？ or \nそもそもtrain_test_splitで上手く分けられていた？
5,6,"ver5に追加\nDAYS_BIRTH+CODE_GENDERを用いて\nM1, F1の層に分ける",細かい年齢の差が、\n返済に影響するとは考えにくい\n層ごとに分けて効果を見る,AUC: 0.7458±0.0016,0.73385,0.74049,少し上がった。\n係数を見るとF3 => F2 => F1の順で返済する傾向にある\n逆にM〇〇は返す傾向が↑に比べ弱い\n~34を~24&~24~34で区切っても良さそう。 => 大学生は返済しにくい
6,7,ver6にM0層(~24)を追加,~24と24~34では\n~24の方が返済する傾向が弱い,AUC: 0.7459±0.0015,0.73414,0.74066,少し上がった。\n係数を見ると意外にも~24歳の方が男女ともに返す傾向にある
7,8,AMT_GOODS_PRICEとAMT_INCOME_TOTAL\nの比を特徴量として追加,収入に対してどれだけの\nローンを組んだか,AUC: 0.7463±0.0016,0.73444,0.74122,少し上がった。\n他にも四則演算を追加しても良さそう
8,9,AMT_GOODS_PRICEとAMT_CREDIT\nの比を特徴量として追加,借金に対してどれだけの\nロームを組もうとしたか,AUC 0.7466±0.0017,0.73499,0.74194,少し上がった。\n他にも四則演算を追加しても良さそう
9,10,AMT_CREDITとAMT_ANNUITY\nの比を特徴量として追加,年金に対してどれだかの\n借金があるか,AUC 0.7467±0.0017,0.73508,0.74191,publicが僅かに落ちた。\n多重共線性の影響？


In [53]:
!python src/module/preprocessing.py --csv_name "application_train.csv"
!python src/module/train.py --parquet_name "application_train.parquet"

starting
read_csv done
preprocessor fit done
df parquet save done
Preprocessor pickle dump done
starting
read parquet done
train done
train_auc 0.7505757420823669
val_auc 0.7529267039854525
save model done


In [54]:
!python src/module/test.py --csv_name "application_test.csv"

starting
read_csv done
pickle load done
predict done
100%|███████████████████████████████████████| 1.26M/1.26M [00:05<00:00, 223kB/s]
Successfully submitted to Home Credit Default Risksumit done! message: 'カテゴリのimpute => ignoreにする'


### ロジスティック回帰係数

In [50]:
df = pd.read_parquet("/Volumes/ex-ssd/kaggle/data/home_default_risk/dataframe/application_train.parquet")
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_nan,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_CAR_nan,FLAG_OWN_REALTY_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_nan,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE_Unaccompanied,NAME_TYPE_SUITE_Family,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_nan,NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Group of people,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_nan,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_nan,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_nan,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_nan,REGION_POPULATION_RELATIVE,YEARS_EMPLOYED,YEARS_REGISTRATION,YEARS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_nan,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_Waiters/barmen staff,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_HR staff,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_nan,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_I

In [51]:
model = pickle.load(open("/Volumes/ex-ssd/kaggle/data/home_default_risk/model/model.pickle", "rb"))
if config.IS_KFOLD_ENSEMBLE:
    coefs = np.zeros((model[0].coef_.shape))
    for i in range(config.N_FOLD):
        coefs += model[i].coef_
    coefs /= config.N_FOLD
else:
    coefs = model.coef_

In [52]:
feature_df = pd.DataFrame(data=coefs.T, index=df.drop(columns=config.IGNORE_COLS+config.EXCLUDE_COLS_AFTER).columns, columns=["coef"])
feature_df["coef_abs"] = np.abs(feature_df.coef)
feature_df.sort_values(by="coef_abs", ascending=False)

,coef,coef_abs
YEARS_EMPLOYED,-0.933739,0.933739
FLAG_EMP_PHONE,-0.775926,0.775926
ORGANIZATION_TYPE_Realtor,0.743124,0.743124
FLAG_OWN_CAR_Y,-0.619405,0.619405
NAME_EDUCATION_TYPE_Academic degree,-0.556561,0.556561
AMT_GOODS_PRICE,-0.552462,0.552462
EXT_SOURCE_3,-0.498336,0.498336
ORGANIZATION_TYPE_Transport: type 3,0.487462,0.487462
ORGANIZATION_TYPE_Industry: type 12,-0.470139,0.470139
ORGANIZATION_TYPE_Legal Services,0.434515,0.434515
